In [54]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, Search
from folium import FeatureGroup
from folium.plugins import MarkerCluster

In [55]:
file_path = 'C:/Users/User/Documents/Berkas Fikar/Skripsi Fikar/Database/Sekolah_Rekomendasi_Bantuan_BOP_PeningkatanDSBCAN3Cluster.csv'
df = pd.read_csv(file_path)

In [37]:
# Bobot perangkingan
bobot = {
    'skor_akreditasi': 29,
    'siswa_total': 23,
    'jumlah_ruangan': 19,
    'wilayah_total': 14,
    'daya_listrik': 5,
    'jumlah_guru': 5,
    'staf_total': 5
}

In [38]:
# Normalisasi data
for col in bobot.keys():
    if col == 'siswa_total':
        df[f'{col}_n'] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    else:
        df[f'{col}_n'] = 1 - (df[col] - df[col].min()) / (df[col].max() - df[col].min())

In [39]:
# Hitung skor akhir
df['skor_rekomendasi'] = (
    df['skor_akreditasi_n'] * bobot['skor_akreditasi'] +
    df['siswa_total_n'] * bobot['siswa_total'] +
    df['jumlah_ruangan_n'] * bobot['jumlah_ruangan'] +
    df['wilayah_total_n'] * bobot['wilayah_total'] +
    df['daya_listrik_n'] * bobot['daya_listrik'] +
    df['jumlah_guru_n'] * bobot['jumlah_guru'] +
    df['staf_total_n'] * bobot['staf_total']
)

In [26]:
# Normalisasi skor akhir ke skala 100
df['skor_rekomendasi'] = (df['skor_rekomendasi'] - df['skor_rekomendasi'].min()) / (df['skor_rekomendasi'].max() - df['skor_rekomendasi'].min()) * 100

In [27]:
# Perangkingan
df = df.sort_values(by='skor_rekomendasi', ascending=False).reset_index(drop=True)
df['peringkat'] = df.index + 1

In [28]:
# Tampilkan semua kolom asli ditambah skor akhir dan peringkat
hasil_rekomendasi = df.copy()

In [29]:
# Tampilkan hasil perangkingan
print(hasil_rekomendasi.head(10))

       No                      Nama Sekolah   BP  Status  siswa_total  \
0  5260.0               UPT SDN 72 BONTOLOE   SD  Negeri        294.0   
1  2097.0    SD INPRES PENGGENTUNGANG UTARA   SD  Negeri        406.0   
2  2242.0            SD INPRES BORONGKALUKU   SD  Negeri        334.0   
3  4072.0         UPT SD NEGERI 149 PINRANG   SD  Negeri        240.0   
4  2227.0               SD NEGERI ANASSAPPU   SD  Negeri        201.0   
5  4475.0                 SD NEGERI 4 ARAWA   SD  Negeri        257.0   
6  7163.0  UPTD SMPN 31 SATAP LALANG TEDONG  SMP  Negeri        150.0   
7  3076.0          SD KRISTEN BUNGIN PASANG   SD  Swasta        146.0   
8  2318.0          SD INPRES BIRING PANTING   SD  Negeri        176.0   
9  2671.0   UPT SD NEGERI 11 BANGKALA BARAT   SD  Negeri        187.0   

   jumlah_guru  staf_total  jumlah_ruangan  daya_listrik  wilayah_total  ...  \
0          8.0         2.0             7.0        5500.0         1965.0  ...   
1         21.0         2.0          

In [40]:
# Simpan hasil ke file CSV
hasil_rekomendasi.to_csv('Sekolah_Rekomendasi_Bantuan_BOP_PeningkatanDSBCAN3Cluster.csv', index=False)

In [56]:
# Buat peta menggunakan Folium
map = folium.Map(location=[-2.5489, 118.0149], zoom_start=8)
marker_cluster = MarkerCluster().add_to(map)

In [57]:
# Filter jenjang pendidikan
jenjang_groups = {
    'SD': folium.FeatureGroup(name='SD').add_to(map),
    'SMP': folium.FeatureGroup(name='SMP').add_to(map),
    'SMA': folium.FeatureGroup(name='SMA').add_to(map),
    'SMK': folium.FeatureGroup(name='SMK').add_to(map)
}

In [59]:
for idx, row in hasil_rekomendasi.iterrows():
    # Lewati jika Lintang atau Bujur kosong
    if pd.isna(row['Lintang']) or pd.isna(row['Bujur']):
        continue

    popup_info = (
        f"Nama Sekolah: {row['Nama Sekolah']}<br>"
        f"Peringkat Rekomendasi: {row['peringkat']}<br>"
        f"Skor Akreditasi: {row['skor_akreditasi']}<br>"
        f"Jumlah Siswa: {row['siswa_total']}<br>"
        f"Luas Wilayah: {row['wilayah_total']}<br>"
        f"Jumlah Ruangan: {row['jumlah_ruangan']}<br>"
        f"Total Staf: {row['staf_total']}<br>"
        f"Total Guru: {row['jumlah_guru']}<br>"
        f"Daya Listrik: {row['daya_listrik']}<br>"
    )

    # Tentukan warna marker berdasarkan index
    if idx <= 2290:
        warna = 'red'
    elif idx <= 6021:
        warna = 'orange'
    else:
        warna = 'green'

    marker = folium.Marker(
        location=[row['Lintang'], row['Bujur']],
        popup=popup_info,
        icon=folium.Icon(color=warna)
    )

    jenjang = row['BP'] if row['BP'] in jenjang_groups else 'SD'
    marker.add_to(jenjang_groups[jenjang])

In [60]:
# Tambahkan fitur pencarian Nama Sekolah
Search(
    layer=marker_cluster,
    search_label='Nama Sekolah',
    placeholder='Cari sekolah...',
    collapsed=False
).add_to(map)

In [61]:
# Tambahkan fitur pencarian Nama Sekolah
Search(
    layer=marker_cluster,
    search_label='peringkat',
    placeholder='Masukkan Peringkat Sekolah',
    collapsed=False
).add_to(map)

In [62]:
# Tambahkan kontrol layer untuk filter jenjang
folium.LayerControl().add_to(map)

In [63]:
# Simpan peta ke file HTML
map.save('Peta_FIX_Rekomendasi_Sekolah_Sulsel.html')
print("Peta Rekomendasi telah disimpan sebagai 'Peta_FIX_Rekomendasi_Sekolah_Sulsel.html'")

Peta Rekomendasi telah disimpan sebagai 'Peta_FIX_Rekomendasi_Sekolah_Sulsel.html'
